In [1]:
from bs4 import *
import time
import requests
import re
import numpy as np
import pandas as pd
import json
import pickle
from pathlib import Path
import sys
import os
sys.path.insert(0,'../../')
import scripts.base
import inspect
import difflib
import warnings
from db_manager import *
from tqdm import tqdm,trange

from datetime import datetime
from collections import defaultdict


In [2]:
local_host = '/Volumes/Seagate Portable Disk/University of Manitoba/Data Science/Datasets/basketball-analytics/'
db = BaseDatabaseManager(local_host)
# db.info()

In [10]:
a = PageHTML('/players/o/olynyke01.html',content_only=False).auto_instance()
a.fetch()
print(str(a))
# a.render()

PlayerIndexHTML('/players/o/olynyke01.html', content_type=online)


In [11]:
href = 'https://www.basketball-reference.com/boxscores/202304280GSW.html'
a = PageHTML(href,content_only=True).auto_instance()
a.fetch()
print(str(a))
# a.render()

BoxscoresHTML('/boxscores/202304280GSW.html', content_type=online)


In [12]:
a.sub_boxscores_hrefs()

['/boxscores/202304280GSW.html',
 '/boxscores/pbp/202304280GSW.html',
 '/boxscores/shot-chart/202304280GSW.html',
 '/boxscores/plus-minus/202304280GSW.html']

In [13]:
b = PageHTML('/boxscores/pbp/202304280GSW.html').auto_instance()
b.fetch()
# b.render()
str(b)

"BoxscoresPbpHTML('/boxscores/pbp/202304280GSW.html', content_type=online)"

In [15]:
b.sub_boxscores_hrefs()

['/boxscores/202304280GSW.html',
 '/boxscores/pbp/202304280GSW.html',
 '/boxscores/shot-chart/202304280GSW.html',
 '/boxscores/plus-minus/202304280GSW.html']

In [1]:
class DatabaseManager(BaseDatabaseManager):
    # Contains functionality that syncs specific folders
    def sync_all_seasons_index(self,downstream=False,pull=True,warn=False):
        # Check for seasons index file, retrieve it if necessary
        sync_results            = SyncResult()
        all_seasons_index_page  = AllSeasonsIndexHTML()
        sync_status             = self.sync(all_seasons_index_page,pull=pull,warn=warn) # sync main index page
        sync_results[sync_status].extend(all_seasons_index_page.href)
        if downstream:
            for season_href in all_seasons_index_page.seasons_hrefs():
                season_sync_results = self.sync_season_index(season_href,downstream=downstream,pull=pull,warn=warn) # sync each season index page
                sync_results.update(season_sync_results)
        return sync_results
    
    def sync_season_index(self,href,downstream=False,pull=True,warn=False):
        sync_results            = SyncResult()
        season_index_page       = SeasonIndexHTML(href)
        sync_status             = self.sync(season_index_page.href,pull=pull,warn=warn) # sync season_index_page
        sync_results[sync_status].extend(season_index_page.href)

        if downstream:
            # sync season schedule
            if season_index_page.schedule_href():
                season_schedule_href            = season_index_page.schedule_href()
                season_schedule_sync_results    = self.sync_season_schedule(season_schedule_href,downstream=downstream,pull=pull,warn=warn) # sync season_schedule_page 
                sync_results.update(season_schedule_sync_results)
            # sync other pages if necessary

        return sync_results
    
    def sync_season_schedule(self,href,downstream=False,pull=True,warn=False):
        sync_results            = SyncResult()
        season_schedule_page    = SeasonScheduleHTML(href)
        sync_status             = self.sync(season_schedule_page.href,pull=pull,warn=warn) # sync season_schedule_page 
        sync_results[sync_status].extend(season_schedule_page.href)

        # sync monthly schedule if available 
        if downstream:
            if season_schedule_page.monthly_schedule_hrefs():
                for monthly_schedule_href in season_schedule_page.monthly_schedule_hrefs():
                    monthly_schedule_page   = SeasonScheduleHTML(monthly_schedule_href)
                    sync_status             = self.sync(monthly_schedule_page,pull=pull,warn=warn) # sync season_schedule_page 
                    sync_results[sync_status].extend(monthly_schedule_page.href)

        return sync_results

    def sync_boxscores():
        pass

    pass


db_path = './demo'
bbref_db = DatabaseManager(db_path)
bbref_db.browse()



NameError: name 'BaseDatabaseManager' is not defined